# Fairness in Social Influence Maximization via Optimal Transport  
### Authors: Guillaume MARIN-BERTIN & Jaishan BURTON ELMO  

**Table of Contents**  
- 1. [Introduction](#introduction)  
- 2. [Related Work](#related-work)  
- 3. [Fairness via Optimal Transport](#fairness-via-optimal-transport)  
- 4. [Evaluation](#evaluation)  
- 5. [Results](#results)  
- 6. [Reproducibility](#reproducibility)  
- 7. [Conclusion](#conclusion)  
- [Appendix](#appendix)  
- [References](#references)  

This is a blog post about the article “Fairness in Social Influence Maximization via Optimal Transport” published by Shubham Chowdhary et al. in 2024 and available [**here**](https://neurips.cc/virtual/2024/poster/94521).

### [1. Introduction](#introduction)  
Social networks have become a crucial vector for spreading information, influencing public opinion, and promoting products or ideas. In this context, the **Influence Maximization (IM)** problem aims to identify the most strategic individuals to initiate a diffusion process and maximize its reach. Applications of IM range from viral marketing to public health campaigns and disinformation detection.  

However, traditional IM algorithms focus solely on maximizing the number of influenced individuals without considering fairness. This can lead to unequal information diffusion, where certain groups are systematically disadvantaged. Addressing this issue is crucial, particularly in sensitive applications such as political campaigns, healthcare awareness, and social justice movements.  

In this blog post, we introduce a new fairness-aware approach to Influence Maximization based on **Optimal Transport (OT)**. Unlike existing methods, our approach ensures that the information spreads in an equitable manner across different demographic groups.

### [2. Related Work](#related-work)

The Social Influence Maximization (SIM) problem has been widely studied, with research efforts primarily focused on optimizing information diffusion in social networks. Existing approaches can be broadly classified into three categories: greedy-based heuristics, fairness-aware methods, and optimal transport-based approaches. 

**Greedy-based heuristics**
Early works, such as Kempe et al. (2003), formulated SIM as a submodular optimization problem, showing that a greedy strategy achieves a \((1 - 1/e)\)-approximation guarantee. Later methods leveraged heuristics like degree centrality, PageRank, and community-based seed selection to improve efficiency. However, these approaches do not account for fairness, potentially reinforcing structural biases in social networks.

**Fairness-aware methods**  
To address fairness concerns, recent studies introduced constraints ensuring more equitable information diffusion across demographic groups. Several fairness criteria have been explored: 

- Equity-based fairness (*Stoica et al., 2020*) ensures that, in expectation, the same proportion of users in each group is reached.  
- Max-min fairness (*Fish et al., 2019; Zhu et al., 2019*) maximizes the minimum probability of a group receiving information to prevent exclusion.  
- Welfare-based approaches (*Rahmattalabi et al., 2021*) optimize outreach under social welfare constraints, while diversity-aware methods (*Tsang et al., 2019*) seek to prevent disproportionate advantages for certain groups.  

However, these approaches rely on marginal outreach probabilities, which fail to capture stochastic variations in diffusion. As a result, they can misclassify unfair scenarios as fair, despite significant disparities in influence spread.  

**Fairness via Optimal Transport**  
To overcome these limitations, we propose a fairness-aware framework based on Optimal Transport (OT). This method introduces a new Mutual Fairness metric and the Stochastic Seed Selection Descent (S3D) algorithm, ensuring a balanced trade-off between fairness and efficiency. Unlike prior methods, our approach explicitly accounts for joint probability distributions, providing a more robust measure of fairness in influence propagation.